<a href="https://colab.research.google.com/github/kodikarthik21/CS6910---Fundamentals-of-Deep-Learning/blob/main/Assignment03/seq2seq_without_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/lexicons.zip'

Archive:  /content/drive/MyDrive/lexicons.zip
   creating: lexicons/
  inflating: lexicons/ta.translit.sampled.test.tsv  
  inflating: lexicons/ta.translit.sampled.dev.tsv  
  inflating: lexicons/ta.translit.sampled.train.tsv  


In [ ]:
!pip install wandb

     |████████████████████████████████| 2.1MB 18.8MB/s 
     |████████████████████████████████| 133kB 52.2MB/s 
     |████████████████████████████████| 102kB 13.8MB/s 
     |████████████████████████████████| 163kB 50.3MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=0d3acbdd345c6f39ef61283708a05f968fdd1ecd3c204293473ff309fff620a2
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=8b4e695015cbd777346c53ba2b61b18808c9919ce1fe6ba6ecf48e0b000c300d
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from wandb.keras import WandbCallback

In [ ]:
train_df = pd.read_csv("/content/lexicons/ta.translit.sampled.train.tsv", sep = '\t', header = None)
train_df = train_df.dropna(axis=0)

val_df = pd.read_csv("/content/lexicons/ta.translit.sampled.dev.tsv", sep = '\t', header = None)
val_df = val_df.dropna(axis=0)

In [ ]:
train_df

,0,1,2
0,ஃபியட்,fiat,2
1,ஃபியட்,phiyat,1
2,ஃபியட்,piyat,1
3,ஃபிரான்ஸ்,firaans,1
4,ஃபிரான்ஸ்,france,2
...,...,...,...
68213,ஹோல்ட்,holtt,1
68214,ஹோல்ட்,hoold,1
68215,ஹோல்ட்,hoolt,1
68216,ஹோல்ட்,hooltt,1


In [ ]:
val_df

,0,1,2
0,ஃபயர்,fire,3
1,ஃபயர்,phayar,1
2,ஃபார்,baar,1
3,ஃபார்,bar,1
4,ஃபார்,far,1
...,...,...,...
6822,ஹோலி,hoolley,1
6823,ஹோலி,hoolli,2
6824,ஹோலி,hoolly,1
6825,ஹோலி,hooly,2


In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for input_text,target_text in zip(train_df[1][0:],train_df[0][0:]):
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

if ' ' not in input_token_index:
  input_token_index[' '] = len(input_token_index)

if ' ' not in target_token_index:
  target_token_index[' '] = len(target_token_index)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters) + 1
num_decoder_tokens = len(target_characters) + 1
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t] = input_token_index[char]
    encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :] = target_token_index[" "]
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 68215
Number of unique input tokens: 27
Number of unique output tokens: 47
Max sequence length for inputs: 30
Max sequence length for outputs: 26


In [ ]:
# Vectorize the data.
val_input_texts = []
val_target_texts = []

for val_input_text,val_target_text in zip(val_df[1][0:],val_df[0][0:]):
    val_input_texts.append(val_input_text)
    val_target_texts.append(val_target_text)

val_encoder_input_data = np.zeros(
    (len(val_input_texts), max_encoder_seq_length), dtype="float32"
)
val_decoder_input_data = np.zeros(
    (len(val_input_texts), max_decoder_seq_length), dtype="float32"
)
val_decoder_target_data = np.zeros(
    (len(val_input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(val_input_texts, val_target_texts)):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t] = input_token_index[char]
    val_encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        val_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    val_decoder_input_data[i, t + 1 :] = target_token_index[" "]
    val_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
def model(cell_name, embedding_size, hidden_size, num_enc_layers, num_dec_layers, dropout, r_dropout, batch_size, num_epochs, optimizer):
  
  cell_type = { "RNN": keras.layers.SimpleRNN,
                "GRU": keras.layers.GRU,
                "LSTM": keras.layers.LSTM
  }

  learning_rate = 0.01
  decay_rate = 0

  optimizers = {"adam": tf.keras.optimizers.Adam(lr=learning_rate, decay=decay_rate),
                "nadam": tf.keras.optimizers.Nadam(lr=learning_rate, decay=decay_rate),
                "rmsprop": tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay_rate),
                "adagrad": tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
  }

  # Define an input sequence and process it.
  encoder_inputs = keras.layers.Input(shape=(None,), name="input_1")
  embedding = keras.layers.Embedding(num_encoder_tokens, embedding_size, name="embedding_1")(encoder_inputs)
  encoder_seq, *encoder_state = cell_type[cell_name](hidden_size, return_sequences=True,return_state = True, name="encoder_1") (embedding)
  for i in range(1, num_enc_layers):
    encoder_seq, *encoder_state = cell_type[cell_name](hidden_size, return_sequences=True,return_state = True, name="encoder_"+str(i+1), dropout=dropout,
                                                       recurrent_dropout=r_dropout) (encoder_seq)

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None,), name="input_2")
  decoder_embedding = keras.layers.Embedding(num_decoder_tokens, embedding_size, name="embedding_2")(decoder_inputs)

  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_seq, *_ = cell_type[cell_name](hidden_size,return_sequences=True, return_state=True, name="decoder_1")(decoder_embedding,initial_state=encoder_state)
  for i in range(1, num_dec_layers):
    decoder_seq, *_ = cell_type[cell_name](hidden_size, return_sequences=True, return_state=True, name="decoder_"+str(i+1),dropout=dropout,
                                      recurrent_dropout=r_dropout) (decoder_seq)

  decoder_outputs = keras.layers.Dense(num_decoder_tokens, activation="softmax", name="dense_1")(decoder_seq)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  model.compile(
    optimizer=optimizers[optimizer], loss="categorical_crossentropy", metrics=["accuracy"]
  )
  model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size=batch_size,
      epochs=num_epochs,
      validation_data=([val_encoder_input_data, val_decoder_input_data],
                      val_decoder_target_data),
      callbacks=[WandbCallback()]
  )

In [ ]:
def swp():

  hyperparameter_defaults = dict(
      cell_name = "LSTM",
      embedding_size = 256,
      hidden_size = 32,
      num_enc_layers = 2,
      num_dec_layers = 2,
      dropout = 0.2,
      r_dropout = 0.2,
      batch_size = 512,
      num_epochs = 10,
      optimizer = "adam"
  )

  wandb.init(project="Assignment 3 without Attention", config=hyperparameter_defaults)
  config = wandb.config
  wandb.run.name = "{}_cell_{}_embSize_{}_hiddenSize_{}_encLayers_{}_decLayers_{}_dropout_{}_rDropout_{}_batchSize_{}_epochs_{}_opt".format(config.cell_name, 
                    config.embedding_size,config.hidden_size, config.num_enc_layers, config.num_dec_layers, config.dropout, config.r_dropout, config.batch_size, config.num_epochs, 
                    config.optimizer)
  
  model(config.cell_name, config.embedding_size,config.hidden_size, config.num_enc_layers, config.num_dec_layers, config.dropout, config.r_dropout, config.batch_size, 
         config.num_epochs, config.optimizer)
  wandb.run.finish()

In [ ]:
sweep_config = {
  "name": "My Sweep",
  "method": "random",
  "project": "Assignment02",
  "metric":{
      "name":"val_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "cell_name": {
            "values": ['RNN', 'LSTM', 'GRU']
        },
        "embedding_size": {
            "values":[16, 32, 64, 128, 256]
        }, 
        "hidden_size": {
            "values":[16, 32, 64, 128, 256]
        },
        "num_enc_layers":{
            "values":[1, 2, 3]
        },
        "num_dec_layers":{
            "values":[1, 2, 3]
        },  
        "dropout":{
            "values":[0, 0.2, 0.3, 0.4]
        },
        "r_dropout": {
            "values":[0, 0.2, 0.3, 0.4]
        },
        "batch_size": {
            "values":[256, 512]
        },
        "num_epochs": {
            "values":[5, 10, 15]
        },
        "optimizer": {
            "values":['adam', 'nadam','rmsprop','adagrad']
        }
    }
}

Create sweep with ID: kgj7mc2s
Sweep URL: https://wandb.ai/kodikarthik21/uncategorized/sweeps/kgj7mc2s


In [ ]:
wandb.agent('xsvxwp4i', swp)

wandb: Agent Starting Run: cvu6b6xw with config:
wandb: 	batch_size: 512
wandb: 	cell_name: RNN
wandb: 	dropout: 0.2
wandb: 	embedding_size: 64
wandb: 	hidden_size: 32
wandb: 	num_dec_layers: 3
wandb: 	num_enc_layers: 3
wandb: 	num_epochs: 10
wandb: 	optimizer: adagrad
wandb: 	r_dropout: 0.2
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/10
134/134 [==============================] - 31s 201ms/step - loss: 2.1083 - accuracy: 0.5760 - val_loss: 1.1867 - val_accuracy: 0.7280
Epoch 2/10
134/134 [==============================] - 26s 192ms/step - loss: 1.2985 - accuracy: 0.7051 - val_loss: 1.0541 - val_accuracy: 0.7461
Epoch 3/10
134/134 [==============================] - 26s 191ms/step - loss: 1.1718 - accuracy: 0.7195 - val_loss: 0.9916 - val_accuracy: 0.7516
Epoch 4/10
134/134 [==============================] - 26s 191ms/step - loss: 1.1013 - accuracy: 0.7256 - val_loss: 0.9482 - val_accuracy: 0.7532
Epoch 5/10
134/134 [==============================] - 26s 190ms/step - loss: 1.0603 - accuracy: 0.7284 - val_loss: 0.9430 - val_accuracy: 0.7537
Epoch 6/10
 11/134 [=>............................] - ETA: 24s - loss: 1.0348 - accuracy: 0.7309